In [1]:
%pylab inline
plt.style.use("bmh")

Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)

In [3]:
import io
import pandas as pd
import requests
import tqdm

# Погода: источники

Для получения погодных данных будет использовать API NOAA (National Centers for Environmental Information). Через этот API можно удобно получить данные сети Global Historical Climatology Network. GHCN агрегирует данные с наземных метеостанций.

При запросе к API NOAA необходимо указать:


- **набор данных**, в данном случае - `daily-summaries`
- **список станций**,
- **тип данных**, которые запрашиваются (мы будем использовать температуру и осадки).

Более подробную информацию об API можно получить на странице [NOAA](https://www.ncei.noaa.gov/support/access-data-service-api-user-documentation).

Выберем по одной станции для Вьетнама, Индонезии и Таиланда (для задачи о прогнозе котировок каучука).

На странице [Data Tools: Find a Station](https://www.ncdc.noaa.gov/cdo-web/datatools/findstation) можно выбрать больше станций. Не для всех из них могут быть доступны необходимые данные, поэтому фильтруйте набор данных (`Select Dataset`), временной промежуток и типы данных (`Data Categories`).

Эти данные вы можете использовать во всех задачах, включая промышленные. Если вы добавляете только переменные или набор данных, то сообщать в чате об этом не нужно.

In [4]:
BASE_URL = "https://www.ncei.noaa.gov/access/services/data/v1"

STATIONS = ["IDM00096087", "VMM00048914", "TH000048426"]
STATIONS_QUERY = f'stations={",".join(STATIONS)}'

DATA_TYPES = ["TAVG", "TMAX", "TMIN", "PRCP"]
DATA_TYPES_QUERY = f'dataTypes={",".join(DATA_TYPES)}'

START = "2002-01-01"
END = "2019-08-01"

BASE_PARAMS = ["units=metric",
               "dataset=daily-summaries",
               f"startDate={START}",
               f"endDate={END}",               
               STATIONS_QUERY,
               DATA_TYPES_QUERY]

In [5]:
response = requests.get("?".join([BASE_URL, "&".join(BASE_PARAMS)]))

In [6]:
assert response.status_code==200

In [7]:
data = pd.read_csv(io.StringIO(response.text), parse_dates=["DATE"])

In [8]:
data.head()

,STATION,DATE,PRCP,TAVG,TMAX,TMIN
0,IDM00096087,2002-01-01,NaN,26.3,NaN,NaN
1,IDM00096087,2002-01-03,NaN,27.3,30.4,23.0
2,IDM00096087,2002-01-04,NaN,25.9,30.2,NaN
3,IDM00096087,2002-01-05,NaN,27.2,30.6,25.1
4,IDM00096087,2002-01-06,NaN,27.1,30.2,25.1


In [9]:
data.groupby("STATION").count()

,DATE,PRCP,TAVG,TMAX,TMIN
STATION,,,,,
IDM00096087,6235,537,6235,5012,4616
TH000048426,6318,3305,6318,5606,4729
VMM00048914,6320,6145,6320,4953,4363


Заполняем пропуски:

In [10]:
data.fillna({"PRCP":0}, inplace=True)
data.ffill(inplace=True)

In [12]:
data.head()

,STATION,DATE,PRCP,TAVG,TMAX,TMIN
0,IDM00096087,2002-01-01,0.0,26.3,NaN,NaN
1,IDM00096087,2002-01-03,0.0,27.3,30.4,23.0
2,IDM00096087,2002-01-04,0.0,25.9,30.2,23.0
3,IDM00096087,2002-01-05,0.0,27.2,30.6,25.1
4,IDM00096087,2002-01-06,0.0,27.1,30.2,25.1


Переформатируем данные:

In [13]:
pd.date_range(START, END, freq="1D")

DatetimeIndex(['2002-01-01', '2002-01-02', '2002-01-03', '2002-01-04',
               '2002-01-05', '2002-01-06', '2002-01-07', '2002-01-08',
               '2002-01-09', '2002-01-10',
               ...
               '2019-07-23', '2019-07-24', '2019-07-25', '2019-07-26',
               '2019-07-27', '2019-07-28', '2019-07-29', '2019-07-30',
               '2019-07-31', '2019-08-01'],
              dtype='datetime64[ns]', length=6422, freq='D')

In [14]:
full_data = []

for station, station_data in data.groupby("STATION"):
    full_data.append(station_data
                     .set_index("DATE")
                     .drop("STATION", axis=1)
                     .rename(lambda cl: f"{cl}_{station}", axis=1))

full_data = pd.concat(full_data, axis=1)
full_data = full_data.reindex(pd.date_range(START, END, freq="1D"))

In [15]:
full_data.fillna({cl:0 for cl in full_data.columns if "PRCP" in cl}, inplace=True)
full_data.ffill(inplace=True)

# Погода: признаки

Признаки для погоды можно построить точно так же, как и другие в задачах о каучуке и ПЭТФ, но добавим `sum` (для осадков этот показатель может быть важен).

In [16]:
PERIODS = ["30D", "90D", "180D"]
AGGREGATES = ["mean", "max", "min", "sum"]

In [17]:
all_features = []

for period in tqdm.tqdm_notebook(PERIODS):
    for agg in AGGREGATES:
        rolling_features = full_data.rolling(period).aggregate(agg)
        rolling_features.rename(lambda x: "_".join([x, period, agg]), axis=1, inplace=True)
        all_features.append(rolling_features[rolling_features.index.day==9])
all_features = pd.concat(all_features, axis=1)

Для ПЭТФ и других задач вычисление будет немного отличаться, но смысл вы поняли:)

In [18]:
all_features = all_features[[cl for cl in all_features.columns if "_sum" not in cl or "PRCP" in cl]]

In [19]:
all_features["prediction_date"] = (all_features.index
                                   + pd.TimedeltaIndex(all_features.index.days_in_month-8,
                                                       unit="D"))
all_features.index.name = "date"

Теперь эти данные можно добавить к исходным рыночным признакам:

In [20]:
all_features.to_csv("rubber_weather.csv")

Несколько подсказок:
    
- использование **нескольких погодных станций** может помочь,
- **температура помогает в задаче о прогнозе котировок каучука**, а вот насколько важны осадки и другие показатели - предстоит выяснить вам. В других задачах погодные данные также могут помочь,
- вы можете подобрать погодные станции **ближе к предполагаемому месту**, в котором происходят важные для соответствующей задачи процессы. Но вам придется самостоятельно выяснить ориентировочные координаты. Например, для задачи о прогнозе котировок каучука важно знать, где растут каучуконосные растерия.

# Данные о производстве, площадях произрастания и т.д.

Натуральный каучук является сельскохозяйственным продуктом. Его выпуск зависит не только от погоды, но и от площади посевов, интенсивности использования и многих других факторов.

Соответствующая информация содержится на порталах открытых данных [Малайзии](http://www.data.gov.my/data/en_US/dataset/), [Индонезии](https://data.go.id/dataset) и [Таиланда](https://data.go.th/). К сожалению, данные не всегда отформатированы качественно и большую часть из них придется привести к подходящему виду вручную. Кроме того, в некоторых случаях описание данных не соответствует содержимому. Что же, придется покопаться:)

Мы бы советовали обратить внимание на следующие (и аналогичные) наборы данных:

- [MALAYSIA : AREA REPLANTED WITH RUBBER](http://www.data.gov.my/data/en_US/dataset/malaysia-area-replanted-with-rubber),
- [INDONESIA AND THAILAND : AREA AND PRODUCTION OF NATURAL RUBBER](http://www.data.gov.my/data/ms_MY/dataset/indonesia-and-thailand-area-and-production-of-natural-rubber-2017).